# How to: Conditions

Conditions are logic which runs on the check results in order to rate it as passing value or not passing.

## Add custom condition

In order to write conditions we first have to know what value a given check proLet's look at the check `DatasetsSizeComparison` and see it's return value in order to write condition for it.

In [3]:
from deepchecks.checks import DatasetsSizeComparison
from deepchecks import Dataset

import pandas as pd

# TODO which data to load
train_dataset = Dataset(pd.DataFrame(data={'x': [1,2,3,4,5,6,7,8,9]}))
test_dataset = Dataset(pd.DataFrame(data={'x': [1,2,3]}))

result = DatasetsSizeComparison().run(train_dataset, test_dataset)
result.value

{'Train': {'Size': 9}, 'Test': {'Size': 3}}

Now we know the return value looks like. let's add a new condition that validates the ratio between train and test sizes is inside a given range.
To create condition we need to use the `add_condition` method of the check which accepts condition name and a function. this function receives the value of the `CheckResult` that we saw above and should return a boolean or `ConditionResult`

In [6]:
# Our parameters for the condition
low_threshold = 0.4
high_threshold = 0.6

# Create the condition function
def condition(value: dict): 
    ratio = value['Test']['Size'] / value['Train']['Size']
    if low_threshold <= ratio <= high_threshold:
        return True
    else:
        return False

# Create the condition name
name = f'Test-Train ratio is between {low_threshold} to {high_threshold}'

# Create check instance with the condition 
check = DatasetsSizeComparison().add_condition(name, condition)

In [8]:
from deepchecks import Suite

# Using suite to run check & condition
suite = Suite('custom suite',
    check
)

suite.run(train_dataset, test_dataset)

custom suite:   0%|          | 0/1 [00:00<?, ? Check/s]

custom suite 
 The suite is composed of various checks such as: Datasets Size Comparison, etc... 
 Each check may contain conditions (which results in 
 ✓ /
 ✖ /
 ! 
 ), as well as other outputs such as plots or tables. 
 Suites, checks and conditions can all be modified (see tutorial [link]). 
 Conditions Summary

Status,Check,Condition,More Info
✖,Datasets Size Comparison,Test-Train ratio is between 0.4 to 0.6,


Additional Outputs

Datasets Size Comparison

Verify Test dataset size comparing it to the Train dataset size.

,Train,Test
Size,9,3


Great! looks like our condition is working. but notice our condition didn't pass, without letting us know why. for that we have option to provide more details when using `ConditionResult`

In [10]:
from deepchecks import ConditionResult

# Our parameters for the condition
low_threshold = 0.4
high_threshold = 0.6

# Create the condition function
def condition(value: dict): 
    ratio = value['Test']['Size'] / value['Train']['Size']
    if low_threshold <= ratio <= high_threshold:
        return ConditionResult(True)
    else:
        return ConditionResult(False, f'Test-Train ratio is {ratio:.2}')

# Create the condition name
name = f'Test-Train ratio is between {low_threshold} to {high_threshold}'

# Create check instance with the condition 
check = DatasetsSizeComparison().add_condition(name, condition)

# Using suite to run check & condition
suite = Suite('custom suite',
    check
)

suite.run(train_dataset, test_dataset)

custom suite:   0%|          | 0/1 [00:00<?, ? Check/s]

custom suite 
 The suite is composed of various checks such as: Datasets Size Comparison, etc... 
 Each check may contain conditions (which results in 
 ✓ /
 ✖ /
 ! 
 ), as well as other outputs such as plots or tables. 
 Suites, checks and conditions can all be modified (see tutorial [link]). 
 Conditions Summary

Status,Check,Condition,More Info
✖,Datasets Size Comparison,Test-Train ratio is between 0.4 to 0.6,Test-Train ratio is 0.33


Additional Outputs

Datasets Size Comparison

Verify Test dataset size comparing it to the Train dataset size.

,Train,Test
Size,9,3


## Remove conditions